<a href="https://colab.research.google.com/github/okeashwini/fly_egg_counting/blob/main/egg_counting_with_flyModel2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pipeline to segment and count fly eggs in images captured by RoboCam using Stardist.

https://pypi.org/project/stardist/

# Install Stardist, import necessary libraries and define functions



In [1]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# install stardist
!pip install stardist

# install skimage
!pip install scikit-image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 kB 10.8 MB/s eta 0:00:00


In [3]:

# load the model:
from stardist.models import StarDist2D
#model = StarDist2D.from_pretrained('2D_versatile_fluo')
#model = StarDist2D(None, name='flyModel2', basedir='/content/drive/MyDrive')
# use model from the shared public folder
model  = StarDist2D(None, name = 'flyModel', basedir='/content/drive/MyDrive/Models/FlyModel')
# import necessary libraries
from stardist.data import test_image_nuclei_2d
from stardist import export_imagej_rois, random_label_cmap, _draw_polygons
lbl_cmap = random_label_cmap()
from stardist.plot import render_label



from csbdeep.utils import normalize
import matplotlib.pyplot as plt
from skimage import io
import cv2
import numpy as np
np.random.seed(6)
import pandas as pd
from csbdeep.utils.tf import keras_import
%matplotlib inline
%config InLineBackend.figure_format = 'retina'

from skimage import data, color
from skimage.transform import hough_circle, hough_circle_peaks
from skimage.feature import canny
from skimage.draw import circle_perimeter
from skimage.util import img_as_ubyte
from skimage.measure import regionprops_table
import math

import glob
from pathlib import Path
import os.path

from PIL import Image, ImageDraw
keras = keras_import()

from natsort import natsorted


Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.705662, nms_thresh=0.3.


In [4]:


## read image and preprocess
def preprocess_image(filename):
  rawImage = io.imread(filename, as_gray=True)
  img8bit = cv2.normalize(rawImage, None, 0, 255, cv2.NORM_MINMAX, dtype = cv2.CV_8U)
  invImg = ~img8bit
  return (invImg)

def empty_image(invImg):
  if np.min(invertedImage) == np.max(invertedImage):
    return(True)
  else:
    return(False)

## generate well mask
def make_well_mask(filename, estRadius):
  # prepreocess image to crop
  #rawImage = io.imread(filename, as_gray=True)
  #img8bit = cv2.normalize(rawImage, None, 0, 255, cv2.NORM_MINMAX, dtype = cv2.CV_8U)
  #invImg = ~img8bit

  wellImg = cv2.imread(filename, 0)
  wellImg = cv2.convertScaleAbs(wellImg, alpha=1.5, beta=0)
  edges = canny(wellImg, sigma=1, low_threshold=10, high_threshold=20)

  # detect radii
  hough_radii = np.arange(estRadius-40, estRadius+40,20)
  hough_res = hough_circle(edges, hough_radii)

  # select the most prominent circle
  accums, cx,cy, radii = hough_circle_peaks(hough_res, hough_radii, total_num_peaks=1)
  h,w= wellImg.shape
  ## Create same size alpha layer with circle
  alpha = Image.new('L', (w,h),0)
  draw = ImageDraw.Draw(alpha)
  # make the circle smaller to exclude the bright edges
  #radii = radii-40
  draw.pieslice([cx-radii,cy-radii,cx+radii,cy+radii],0,360,fill=255)

  #Convert alpha Image to numpy array
  wellMask=np.array(alpha)

  return(wellMask, int(cx),int(cy), int(radii))


## run segmentation model and filter eggs by size
def run_segmentation(cropped, probThreshold, cx, cy, radius):
  labels, details = model.predict_instances(normalize(cropped), prob_thresh=probThreshold )
  allObjects = regionprops_table(labels, invertedImage, properties = ['label', 'area', 'centroid','bbox','euler_number','eccentricity','solidity', 'mean_intensity','min_intensity','max_intensity'] )
  eggs = pd.DataFrame(allObjects)
  # calculate position from center
  eggs['position'] = eggs.apply(lambda x: math.dist([x['centroid-0'],x['centroid-1']], [cy,cx]), axis=1)
  # filter by intensity
  #eggs = eggs[eggs['mean_intensity'].between(100, 220)]
  # filter by size
  eggs = eggs[eggs['area'].between(2500, 4000)]

  # filter by position
  eggs = eggs[eggs['position']<(radius-30)]
  #eggs = eggs[eggs['position']<(radius)]
  # change the labels
  eggs['label'] = range(0,len(eggs))
  return(labels, eggs)


# make output images
def generateOutput(invImg, eggList, cx, cy, radius):
  eggBoxes = np.copy(invImg)
  eggNumbers = np.copy(invImg)
  for index,row in eggList.iterrows():
    start_point = (int(row['bbox-1']), int(row['bbox-0']))
    end_point = (int(row['bbox-3']), int(row['bbox-2']))
    cv2.rectangle(eggBoxes, start_point, end_point, color=(255,255,255), thickness=6)
    eggID = int(row['label'])+1
    #area = int(row['area'])
    #eggID = int(row['mean_intensity'])
    cv2.putText(eggNumbers, f'{eggID}', (int(row['centroid-1'])+2, int(row['centroid-0'])+1), fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale = 2, color = (255, 255, 255), thickness=6)
    #cv2.putText(eggNumbers, f'{area}', (int(row['centroid-1'])+2, int(row['centroid-0'])+1), fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale = 2, color = (255, 255, 255), thickness=6)


  numberOfEggs = int(len(eggList))
  cv2.putText(eggBoxes, f'Egg count: {numberOfEggs}', (2500,250), fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale = 5, color = (255, 255, 255), thickness=8)
  cv2.putText(eggNumbers, f'Egg count: {numberOfEggs}', (2500,250), fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale = 5, color = (255, 255, 255), thickness=8)
  # add cropped well circle
  cv2.circle(eggBoxes, (cx,cy), radius, (255,255,255), 6)
  cv2.circle(eggNumbers, (cx,cy), radius, (255,255,255), 6)
  return(eggBoxes, eggNumbers)







In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# File handling and inputs

In [6]:

#@title Enter full path to the folder with images
mainPath = "/content/drive/MyDrive/fly_egg_counting_Stardist"
ExperimentName = "Chem3_Day7_Rep7_Plate2" #@param {type:"string"}
pathToFolder = f'{mainPath}/images/{ExperimentName}/'
outPath = f'{mainPath}/predictions_flyModel2/{ExperimentName}/'
estimatedWellDiameter = 2200 #@param {type:"integer"}

# create the output folder
if not os.path.exists(outPath):
  os.makedirs(outPath)

filenames = natsorted(glob.glob(f'{pathToFolder}*.jpg'))
print(len(filenames))



63


# Image processing



*   Preprocess
*   Crop well
*   Segment objects
*   Filter egg objects
*   Output data





In [7]:

eggCounts = pd.DataFrame(columns=['Filename','EggCount'])

well = False

for filename in filenames:
  basename = f'{Path(filename).stem}.jpg'
  rawImage = io.imread(filename)
  print(f'\nReading file {basename}')
  # preprocess
  invertedImage = preprocess_image(filename)
  print("Preprocessing done!")

  # if image is empty, move to next image
  if empty_image(invertedImage):
    print("Image is empty. Skipping this image.")
    continue

  # detect well for the first image
  if not well:
    wellMask, cx, cy, radius = make_well_mask(filename, estimatedWellDiameter/2)
   #croppedImage = cv2.bitwise_and(invertedImage,invertedImage,mask = wellMask)
    print("Well detection done!")

    radius = int(estimatedWellDiameter/2)
    well = True

  # segment objects and filter eggs by size
  labels, eggs = run_segmentation(invertedImage, probThreshold=0.7, cx=cx, cy=cy, radius=radius)
  print("Segmentation done!")

  # output data
  eggBoxes, eggNumbers = generateOutput(invertedImage, eggs, cx=cx, cy=cy, radius=radius)
  tmpDf = pd.DataFrame([{'Filename':basename, 'EggCount': len(eggs)}])
  eggCounts = pd.concat([eggCounts, tmpDf])
  print(f'Counted {len(eggs)} eggs')

  # save files
  Boxedfilename = f'{outPath}eggs_boxed_{basename}'
  Numfilename = f'{outPath}eggs_numbered_{basename}'
  Croppedfilename = f'{outPath}well_cropped_{basename}'
  InvertedFilename = f'{outPath}well_inverted_{basename}'

  boxedImg, numberedImg = generateOutput(invertedImage, eggs, cx=cx, cy=cy, radius=radius)
  renderedImage = render_label(labels, img=invertedImage)

  cv2.imwrite(Boxedfilename, boxedImg)
  cv2.imwrite(Numfilename, numberedImg)
  cv2.imwrite(InvertedFilename, invertedImage)

eggCounts.to_excel(f'{outPath}egg_counts_{ExperimentName}.xlsx')




Reading file well_1_2024-02-20_120735_.jpg
Preprocessing done!


<ipython-input-4-61337366b9cd> (42): Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)


Well detection done!
Segmentation done!
Counted 26 eggs

Reading file well_2_2024-02-20_120213_.jpg
Preprocessing done!
Segmentation done!
Counted 52 eggs

Reading file well_2_2024-02-20_120741_.jpg
Preprocessing done!
Segmentation done!
Counted 52 eggs

Reading file well_3_2024-02-20_120219_.jpg
Preprocessing done!
Segmentation done!
Counted 41 eggs

Reading file well_3_2024-02-20_120746_.jpg
Preprocessing done!
Segmentation done!
Counted 39 eggs

Reading file well_4_2024-02-20_120225_.jpg
Preprocessing done!
Segmentation done!
Counted 51 eggs

Reading file well_4_2024-02-20_120752_.jpg
Preprocessing done!
Segmentation done!
Counted 52 eggs

Reading file well_5_2024-02-20_120230_.jpg
Preprocessing done!
Segmentation done!
Counted 13 eggs

Reading file well_5_2024-02-20_120757_.jpg
Preprocessing done!
Segmentation done!
Counted 14 eggs

Reading file well_6_2024-02-20_120236_.jpg
Preprocessing done!
Segmentation done!
Counted 6 eggs

Reading file well_6_2024-02-20_120803_.jpg
Preprocess